In [1]:
import numpy as np
import pandas as pd
import pymongo
from bson.objectid import ObjectId
from math import isnan
DATABASE_URL = "mongodb://localhost:27017"
DATABASE_NAME = "mojeWideloDb"
USER_COLLECTION_NAME = "users"
VIDEO_COLLECTION_NAME = "videoMetadata"
REACTION_COLLECTION_NAME = "reactions"
SUBSCRIPTION_COLLECTION_NAME = "subscriptions"
HISTORY_COLLECTION_NAME = "history"


In [ ]:
reaction_collection

In [2]:
db_client = pymongo.MongoClient(DATABASE_URL)
db = db_client[DATABASE_NAME]
video_collection = pd.DataFrame(list(db[VIDEO_COLLECTION_NAME].find()))
user_collection = pd.DataFrame(list(db[USER_COLLECTION_NAME].find()))
reaction_collection = pd.DataFrame(list(db[REACTION_COLLECTION_NAME].find()))
subscription_collection = pd.DataFrame(list(db[SUBSCRIPTION_COLLECTION_NAME].find()))
history_collection = pd.DataFrame(list(db[HISTORY_COLLECTION_NAME].find()))

In [4]:
# merge users with histories
users_history = history_collection.merge(user_collection, on="_id")

# process one user history
user_id = ObjectId("64762699d313339852219002")
user_history_index = history_collection["WatchedVideos"][history_collection["_id"] == user_id].index[0]
# todo: do not use subscriptions? we already implemented subs page... (maybe hide subscribed channels?)
user_subscriptions = subscription_collection[subscription_collection["SubscriberId"] == user_id]
user_reactions = reaction_collection[reaction_collection["UserId"] == user_id]
watched_video = pd.DataFrame(history_collection["WatchedVideos"][user_history_index].copy())
user_watched_videos = watched_video.merge(video_collection, left_on="VideoId", right_on="_id").merge(user_reactions, left_on="VideoId", right_on="VideoId", how="left").merge(user_subscriptions, left_on="AuthorId", right_on="CreatorId", how="left").loc[:,["VideoId", "Date", "Tags_x", "ReactionType","SubscriberId"]]
user_watched_videos["Subscribes"] = user_watched_videos.loc[:,"SubscriberId"].transform(lambda x: not pd.isnull(x))
user_watched_videos = user_watched_videos.drop(columns=["SubscriberId"])
user_watched_videos

KeyError: "['Tags_x'] not in index"

In [22]:
video_count = len(video_collection["AuthorId"])
movie_matrix = np.zeros((video_count, video_count), float)
for i in range(video_count):
    for j in range(video_count):
        tags1, tags2 = video_collection["Tags"][i], video_collection["Tags"][j]
        min_length = (min(len(tags1), len(tags2)))
        if(i == j or min_length == 0):
            print(min_length)
            continue
        movie_matrix[i,j] = len(np.intersect1d(tags1, tags2)) / min_length
movie_matrix

5
5
5


array([[0. , 0.2, 0.2],
       [0.2, 0. , 0. ],
       [0.2, 0. , 0. ]])

In [24]:
from app.const import HISTORY_DEPTH, DEPTH_PENALTY


user_history = watched_video.drop_duplicates(subset='VideoId', keep="last").reset_index()
recommended = pd.DataFrame(columns=["VideoId", "score"])
history_length = len(user_history)
for i in min(range(history_length), HISTORY_DEPTH):
    video_index = video_collection[video_collection["_id"] == user_history["VideoId"][i]].index
    for j in range(len(movie_matrix)):
        if(movie_matrix[video_index, j] > 0):
            recommended.loc[len(recommended)] = {"VideoId": user_history["VideoId"][i], "score": movie_matrix[video_index, j] / (i * DEPTH_PENALTY)}

recommended.drop_duplicates(subset="VideoId", keep="first").sort_values(by="score", ascending=False)


ModuleNotFoundError: No module named 'app'

In [8]:
history_collection["WatchedVideos"][1]

[{'VideoId': ObjectId('647627b6d313339852219003'),
  'Date': datetime.datetime(2023, 5, 30, 17, 40, 51, 718000)},
 {'VideoId': ObjectId('647627b6d313339852219003'),
  'Date': datetime.datetime(2023, 5, 30, 17, 40, 51, 788000)},
 {'VideoId': ObjectId('647627b6d313339852219003'),
  'Date': datetime.datetime(2023, 5, 30, 17, 40, 51, 809000)},
 {'VideoId': ObjectId('647627b6d313339852219003'),
  'Date': datetime.datetime(2023, 5, 30, 17, 41, 1, 821000)},
 {'VideoId': ObjectId('647627b6d313339852219003'),
  'Date': datetime.datetime(2023, 5, 30, 17, 41, 1, 841000)},
 {'VideoId': ObjectId('647627b6d313339852219003'),
  'Date': datetime.datetime(2023, 5, 30, 17, 41, 1, 863000)},
 {'VideoId': ObjectId('64762809d313339852219004'),
  'Date': datetime.datetime(2023, 5, 30, 17, 41, 3, 483000)},
 {'VideoId': ObjectId('647627b6d313339852219003'),
  'Date': datetime.datetime(2023, 5, 30, 17, 41, 36, 305000)},
 {'VideoId': ObjectId('647627b6d313339852219003'),
  'Date': datetime.datetime(2023, 5, 30, 